# PM2.5 en 4 ciudades iberoamericanas — Mapa (estático y animado) + Línea temporal

Este notebook genera las dos visualizaciones propuestas:
1. **Mapa interactivo con burbujas** (estático y **animado por día**).
2. **Línea temporal por ciudad** con banda guía de la OMS.

**Dataset**: `../data/pm25_90d_daily.csv` (promedio diario por ciudad, incluye `lat` y `lon`).

> Sugerencia para el post de Instagram: usa el mapa animado como video/gif y la línea temporal como imagen con anotaciones.

## 0) Cargar datos

In [ ]:
import pandas as pd
import plotly.express as px

# Cargar dataset diario
df = pd.read_csv("../data/pm25_90d_daily.csv", parse_dates=["date"])

# Sanidad mínima
assert {"city","date","pm25","lat","lon"}.issubset(df.columns), "Faltan columnas requeridas."

# Ordenar por fecha por prolijidad
df = df.sort_values("date").reset_index(drop=True)

df.head()

## 1) Mapa interactivo (estático) — Promedio del período por ciudad

In [ ]:
# Promedio del período y coordenadas representativas (mediana)
mean_by_city = (df.groupby('city', as_index=False)
                  .agg(pm25_mean=('pm25','mean'),
                       lat=('lat','median'),
                       lon=('lon','median')))

fig_map = px.scatter_geo(
    mean_by_city,
    lat="lat", lon="lon",
    size="pm25_mean", color="pm25_mean",
    hover_name="city",
    projection="natural earth",
    title="PM2.5 promedio (últimos 90 días) — por ciudad",
)
fig_map.update_traces(marker=dict(opacity=0.9))
fig_map.update_layout(legend_title_text="PM2.5 (µg/m³)")
fig_map.show()

# Exportar a HTML para compartir/hostear
fig_map.write_html("../data/mapa_pm25_promedio.html", include_plotlyjs="cdn")
print("Exportado: ../data/mapa_pm25_promedio.html")

## 2) Mapa interactivo **animado** — Evolución diaria (slider por fecha)

In [ ]:
# Para animación, usamos las mediciones diarias tal cual (una por ciudad y día)
# Nota: la animación puede ser pesada si hay muchos días; es ideal para exportar como video corto
fig_map_anim = px.scatter_geo(
    df,
    lat="lat", lon="lon",
    size="pm25", color="pm25",
    hover_name="city",
    animation_frame=df["date"].dt.strftime("%Y-%m-%d"),
    projection="natural earth",
    title="PM2.5 diario por ciudad — animación (últimos 90 días)",
)
fig_map_anim.update_traces(marker=dict(opacity=0.85))
fig_map_anim.update_layout(legend_title_text="PM2.5 (µg/m³)")
fig_map_anim.show()

# Exportar animado (HTML interactivo con slider)
fig_map_anim.write_html("../data/mapa_pm25_animado.html", include_plotlyjs="cdn")
print("Exportado: ../data/mapa_pm25_animado.html")

## 3) Línea temporal por ciudad + banda OMS

In [ ]:
fig_line = px.line(
    df,
    x="date", y="pm25", color="city",
    labels={"pm25":"PM2.5 (µg/m³)", "date":"Fecha"},
    title="Evolución diaria de PM2.5 (últimos 90 días)"
)

# Banda de seguridad OMS (24h ≈ 15 µg/m³): sombreamos de 0 a 15
fig_line.add_hrect(y0=0, y1=15, line_width=0, fillcolor="LightGreen", opacity=0.2,
                   annotation_text="Zona recomendada OMS (≈15 µg/m³)", annotation_position="top left")

fig_line.show()

# Exportar HTML
fig_line.write_html("../data/serie_pm25.html", include_plotlyjs="cdn")
print("Exportado: ../data/serie_pm25.html")

## 4) (Opcional) Exportar imágenes estáticas para Instagram

In [ ]:
# Si quieres PNG de alta resolución, instala kaleido:
# !pip install -U kaleido
#
# fig_map.write_image("../data/mapa_pm25_promedio.png", scale=3)
# fig_line.write_image("../data/serie_pm25.png", scale=3)
# print("Exportado: ../data/mapa_pm25_promedio.png y ../data/serie_pm25.png")